# Assignment 6: Distributed Systems

Please read the tasks description carefully and implement **only** what the tasks ask you to implement. Closely following the task descriptions will be beneficial, so keep your divergence in check - the test cases below each input cell are the gold standard. Finally, for this assignment, you do not need any error handling, you can assume that all input to your function will be valid.

As for the other assignments, using `print` is encouraged to test your implementation but is never required. Make sure not to confuse `return` and `print` statements: If your function has to **return** something, use the `return` statement. 

Try to implement the tasks yourself or in a small team. If you blindly copy a solution from the Internet or other students, you will not take home any learnings. Rather, make an effort to understand the solution! Furthermore, do not modify the _test cells_ - if you do, you effectively cheat the system which is not helpful for your learning process.

Some aspects of this assignment require you to <strong>self-study</strong> and do some research beyond the lecture contents - use your favorite search engine to look up documentation, usage examples, and definitions of the mentioned functions. There might be tasks where you have to read and investigate the [Python Standard Library](https://docs.python.org/3/library/) to find the documentation for a function that is used or that you want to use.

This assignment will use the third-party module [requests](https://pypi.org/project/requests/). It will also use [json](https://docs.python.org/3/library/json.html) for converting JSON objects into Python dictionaries.

> At the end of this assignment we provide you with a [List](#apilist) of public APIs and API directories. These are not required for this assignment but might be helpful towards your group project. Have a Look at them if your are interested! 


## Task 1: The Networking Stack and HTTP - Self-Study
In this task, you will get some practical experience on top of the theoretical knowledge about networks that were discussed during the lectures. Make sure that you are able to answer the following questions - they will prepare you for Task 2 of this assignment and the final exam.

> **We will not provide a solution for this Task. It is meant as a repitition of what Simon showed you in the lecture.**

- **Question 1**: What is the purpose of the Transmission Control Protocol (TCP)? What is the purpose of the sequence numbers that are embedded in TCP packets?
- **Question 2**: Install and use the [Wireshark](https://www.wireshark.org/) to analyze the network traffic of your computer. Just like Simon showed in the lecture, start recording your "main network interface" that is connected to the Internet (e.g., en0, Wi-Fi, etc.) with Wireshark and open http://example.com in your browser, then stop the Wireshark recording. In Wireshark, identify the HTTP packet your machine sent to the remote server example.com (IP address: `93.184.215.14`) by applying a filter `ip.addr == 93.184.215.14`. Click that packet to see its details, which are displayed conveniently split by networking layer, and find answers to the following questions:
    - **Network Layer (IP)**:
        - **Question 2.1**: Which version of the IP protocol was used?
        - **Question 2.2**: What is the source IP address (i.e. your machine's IP) and what is the destination IP address?
    - **Transport Layer (TCP)**:
        - **Question 2.3**: What local port did your Web browser use to communicate with the server that hosts `example.com`?
        - **Question 2.4**: At what port does the remote server listen to incoming requests?
        
    - **Application Layer (HTTP)**:
        - **Question 2.5**: In the packet, what is the payload that your machine sent to the remote server?
        - **Question 2.6**: Which version of the HTTP protocol was used to send this payload?
        - **Question 2.7**: What is the value of the Content-Type header?
        - **Question 2.8**: What is the name of the HTTP Method that your browser used to retrieve data from `example.com`?
- **Question 3**: From the raw HTTP response of the previous task, note the meaning and the values of the following parameters:
    - The HTTP Status Code
    - The Content-Type header
    - The Content-Length header
- **Question 4**: The response to your request in the previous exercise most probably includes an HTTP status code of 200 OK. Look up the semantics of the following HTTP status codes:
    - 304 Not Modified
    - 401 Unauthorized
    - 404 Not Found
    - 418 I'm a Teapot
- **Question 5**: Finally, clear the filter ip.addr == 93.184.216.34 from Wireshark, and request the secure Website https://unisg.ch in your browser. You should now observe lots of packets that use the TLS protocol to communicate with the server hosting that website. Try to read the payload of any of these packets! Why can you not find any HTTP packets from your interaction with https://unisg.ch in the traffic?
    


## Task 2: Build a Simple Web Mashup
In this task, you will implement a Web Mashup by combining the functionalities of two different Web APIs. Your mashup will use user trip details (origin location, arrival location, datetime of the trip) and will suggest restaurants close to the arrival location based on the trip datetime and user preferences (e.g. radius).

**Learning objectives:**
- Using API keys for authenticating HTTP requests
- Sending HTTP requests
- Defining request parameters
- Constructing HTTP headers
- Analyzing JSON response payloads
- Combining the functionality of two or more Web APIs for building a Web application

_There are [Hints](#hints) at the bottom of this notebook._

In [ ]:
# Required imports

# Handling dates and times
from datetime import datetime
import dateutil

# Force the Europe/Zurich timezone
import time, os
os.environ['TZ'] = 'Europe/Zurich'
# Uncomment the following line ↓ ONLY if you use a unix-based OS (macOS or Linux)
# time.tzset()

In [ ]:
from datetime import datetime
import dateutil

import requests
import json

class Connection:
    '''Connection class
       A class that holds information about the transport means (e.g. the trains) and the times of a connection
    '''
    def __init__(self, destination_x, destination_y, departure, arrival, transport_means):
        '''
        Connection constructor
        :param destination_x: the latitude of the destination
        :param destination_y: the longitude of the destination
        :param departure: a string containing the datetime for the departure
        :param arrival: a string containing the datetime for the arrival
        :param transport_means: a list of the transport means of the connection (e.g. ['IC 5'])
        '''

        if (isinstance(destination_x,float) and isinstance(destination_y,float) and
            isinstance(departure,str) and isinstance(arrival,str) and
            isinstance(transport_means,list)):
            self.destination_x = destination_x
            self.destination_y = destination_y
            self.transport_means = transport_means

            self.departure_time = dateutil.parser.parse(departure.split('+')[0])
            self.arrival_time = dateutil.parser.parse(arrival.split('+')[0])
        else:
            raise AttributeError

    def __str__(self):
        return "{}: {}->{}".format(self.transport_means, self.departure_time, self.arrival_time)

    def get_unix_departure_time(self):
        '''
        Method get_Unix_departure_time
        Returns the local time of departure as a Unix timestamp
        '''
        return int((self.departure_time - datetime(1970,1,1)).total_seconds())

    def get_unix_arrival_time(self):
        '''
        Method get_Unix_arrival_time
        Returns the local time of arrival as a Unix timestamp
        '''
        return int((self.arrival_time - datetime(1970,1,1)).total_seconds())

### Task 2.1: Find the first connection from an origin location to a destination at a specified departure time
    
**GOAL: Use the Swiss public Transport API to get the first connection between two locations at a specified departure time.**

The [**Web API of the Swiss public Transport system**](https://transport.opendata.ch/) provides real-time information about the schedule of many Swiss public transport providers. The Transport API builds on REST-style resources which send JSON-formatted responses.

Based on the [**documentation of the Transport API**](https://transport.opendata.ch/docs.html#connections), we can send an **HTTP GET request** to http://transport.opendata.ch/v1/connections with the following **request parameters** and retrieve the upcoming connections between St. Gallen and Zurich. There, the departure time and date are not specified:

```json
{
    "from": "StGallen",
    "to": "Zurich"
}
```

If there are any upcoming connections from St. Gallen to Zurich, the JSON **response payload** will have the following structure:

```json
{
    "connections": [
        {
            "from": {
                "station": {
                    "id": "8506302",
                    "name": "St. Gallen",
                    "score": null,
                    "coordinate": {
                        "type": "WGS84",
                        "x": 47.42318,
                        "y": 9.369902
                    },
                    "distance": null
                },
                "arrival": null,
                "arrivalTimestamp": null,
                "departure": "2025-03-25T11:17:00+0100",
                "departureTimestamp": 1648203420,
                "delay": 0,
                "platform": "1",
                "prognosis": {
                    "platform": null,
                    "arrival": null,
                    "departure": "2025-03-25T11:17:00+0100",
                    "capacity1st": null,
                    "capacity2nd": null
                },
                "realtimeAvailability": null,
                "location": {
                    "id": "8506302",
                    "name": "St. Gallen",
                    "score": null,
                    "coordinate": {
                        "type": "WGS84",
                        "x": 47.42318,
                        "y": 9.369902
                    },
                    "distance": null
                }
            },
            "to": {
                "station": {
                    "id": "8503000",
                    "name": "Zürich HB",
                    "score": null,
                    "coordinate": {
                        "type": "WGS84",
                        "x": 47.377847,
                        "y": 8.540502
                    },
                    "distance": null
                },
                "arrival": "2025-03-25T12:42:00+0100",
                "arrivalTimestamp": 1648208520,
                "departure": null,
                "departureTimestamp": null,
                "delay": null,
                "platform": "41/42",
                "prognosis": {
                    "platform": null,
                    "arrival": null,
                    "departure": null,
                    "capacity1st": null,
                    "capacity2nd": null
                },
                "realtimeAvailability": null,
                "location": {
                    "id": "8503000",
                    "name": "Zürich HB",
                    "score": null,
                    "coordinate": {
                        "type": "WGS84",
                        "x": 47.377847,
                        "y": 8.540502
                    },
                    "distance": null
                }
            },
            "duration": "00d01:25:00",
            "transfers": 2,
            "service": null,
            "products": [
                "S1",
                "S35",
                "S12"
            ],
            "capacity1st": null,
            "capacity2nd": null,
            "sections": [
                {
                    "journey": {
                        "name": "011138",
                        "category": "S",
                        "subcategory": null,
                        "categoryCode": null,
                        "number": "1",
                        "operator": "THURBO",
                        "to": "Wil SG",
                        "passList": [
                            {
                                "station": {...
                                },
                                .
                                .
                                .
                            },
                            {
                                "station": {...
                                },
                                .
                                .
                                .
                            },
                            .
                            .
                            .
                        ]
                    }
                    .
                    .
                    .
                }
            ]
        }
    ]
}
```

**STEPS:**

Implement a **function** `find_connection` with the following parameters: 
- `origin`: origin location (`string`)
- `destination`: arrival location (`string`)
- `departure_date` : date of departure, in the format YYYY-MM-DD (`string`) 
- `departure_time` : time of departure, in the format in hh:mm (`string`)

The function should send a request to the Swiss Public Transport API for retrieving the connections from `origin` to `destination` at the `departure_date` and `departure_time`.

The function should use the retrieved connections and return a `Connection` object based on the **latitude of the arrival location**, the **longitude of the arrival location**, the **departure**, the **arrival** and the **transport means** (products) of only the 1st retrieved connection.
    
You can find more hints at the end of this document.



In [ ]:
# Add your code here:




In [ ]:
# Run this cell to try out your function
now = datetime.now()
depart_date = now.strftime("%Y-%m-%d")
depart_time = now.strftime("%H:%M")

# Call the function to get the latitude, longitude and timestamp of the arrival (for the 1st connection from St. Gallen to Zurich right now)
first_conn = find_connection("St.Gallen", "Zurich", depart_date, depart_time) 
print(first_conn)

In [ ]:
# Test Cell.
from unittest import TestCase
from inspect import signature
from datetime import datetime

__ = TestCase()

# Signature
__.assertTrue("find_connection" in locals(), msg='You did not define a function called find_connection.')
__.assertEqual(len(signature(find_connection).parameters), 4, msg='Your function does not have exactly 4 parameters.')

t21_return = find_connection("St.Gallen", "Zurich", "2025-04-01", "18:00")
__.assertIsInstance(t21_return, Connection, msg=f"Your function does not return a Connection object, it returns a value of {type(t21_return)}. Check again what your function returns.")
__.assertEqual(t21_return.destination_x, 47.377847, msg=f"The latitude of the Zurich station (destination_x) should be 47.377847.")
__.assertEqual(t21_return.destination_y, 8.540502, msg=f"The longitude of the Zurich station (destination_y) should be 8.540502.")
__.assertTrue(t21_return.departure_time < t21_return.arrival_time, msg="The departure time should be before the arrival time.")
__.assertEqual(t21_return.departure_time, datetime(2025, 4, 1, 18, 7), msg='Your function does not return the right departure time.')
__.assertEqual(t21_return.arrival_time, datetime(2025, 4, 1, 19, 21), msg='Your function does not return the right arrival time.')

print("\n\033[37;42;2m  Success! Your code works as intended.  \033[0m\n")

### Task 2.2: Find open restaurants close to a location

**GOAL: Use the Yelp Fusion API with an API key to get a list of restaurants that are a) open at a specific time and b) within a specific radius of a given geographical location.**

#### HTTP security

The [**Yelp Fusion API**](https://www.yelp.com/developers/documentation/v3/get_started) is a Web API that provides information about local businesses, events and user reviews across 32 countries. In comparison to the public Transport API of Task 2.2, The Yelp Fusion API is private and requires **user authentication**. The HTTP Protocol [defines different Authentication Schemes](https://www.iana.org/assignments/http-authschemes/http-authschemes.xhtml). The Yelp Fusion API uses one of them, namely the **Bearer Authentication Scheme** which involves security tokens called **bearer tokens: The API gives access to the bearers of such tokens.** Based on the [Yelp Authentication Guide](https://www.yelp.com/developers/documentation/v3/authentication), in order to use the API, we need to:
1. [Register](https://www.yelp.com/signup) to Yelp and [request a private API key](https://www.yelp.com/developers/v3/manage_app). (Note: The second link will work only if you logged in to Yelp).
2. Set the value of "Authorization" to `Bearer API_KEY_VALUE` in the **HTTP header of any HTTP requests**. This way the API calls will get authenticated with the API key.

Once we set up the **HTTP header** of a request, we can proceed by defining the **request parameters**. For example, we can send an **HTTP GET request** to https://api.yelp.com/v3/businesses/search to search for businesses with the following **parameters**: 
```
{
    "latitude": 47.42318 ,
    "longitude": 9.369902,
    "radius": 500,
    "categories": "collegeuniv"
}
```

Based on the [documentation of the Yelp Fusion API](https://www.yelp.com/developers/documentation/v3/business_search) the above **request parameters** specify that we want to get the [colleges and universities](https://www.yelp.com/developers/documentation/v3/all_category_list) in St. Gallen within a radius of 500m. The structure of the **response payload** will be similar to this:

```json
{
    "businesses": [
        {
            "id": "r7TVzj6LrTHOfov0CwJaqg",
            "alias": "fhs-st-gallen-hochschule-für-angewandte-wissenschaften-st-gallen",
            "name": "FHS St. Gallen Hochschule für Angewandte Wissenschaften",
            "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/Z2OWdwjOzQwN2rlEy6Rsmg/o.jpg",
            "is_closed": false,
            "url": "https://www.yelp.com/biz/fhs-st-gallen-hochschule-f%C3%BCr-angewandte-wissenschaften-st-gallen?adjust_creative=1Qgj6QZinP_l0CKVRR4IRg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=1Qgj6QZinP_l0CKVRR4IRg",
            "review_count": 1,
            "categories": [
                {
                    "alias": "collegeuniv",
                    "title": "Colleges & Universities"
                }
            ],
            "rating": 4.0,
            "coordinates": {
                "latitude": 47.42541,
                "longitude": 9.37135
            },
            "transactions": [],
            "location": {
                "address1": "Rosenbergstrasse 22",
                "address2": null,
                "address3": null,
                "city": "St. Gallen",
                "zip_code": "9000",
                "country": "CH",
                "state": "SG",
                "display_address": [
                    "Rosenbergstrasse 22",
                    "9000 St. Gallen",
                    "Switzerland"
                ]
            },
            "phone": "+41800800988",
            "display_phone": "+41 800 800 988",
            "distance": 271.1439203578532
        },
        {
            "id": "G5qnNm5pLI3ljpQOYDYt5g",
            "alias": "university-of-st-gallen-saint-gallen",
            "name": "University of St.Gallen",
            "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/wRB_VLyCFQMIRLJYE5BGnw/o.jpg",
            "is_closed": false,
            "url": "https://www.yelp.com/biz/university-of-st-gallen-saint-gallen?adjust_creative=1Qgj6QZinP_l0CKVRR4IRg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=1Qgj6QZinP_l0CKVRR4IRg",
            "review_count": 1,
            "categories": [
                {
                    "alias": "collegeuniv",
                    "title": "Colleges & Universities"
                }
            ],
            "rating": 4.0,
            "coordinates": {
                "latitude": 47.4273277,
                "longitude": 9.3736067
            },
            "transactions": [],
            "location": {
                "address1": "Blumenbergplatz",
                "address2": "",
                "address3": "",
                "city": "Saint Gallen",
                "zip_code": "9000",
                "country": "CH",
                "state": "SG",
                "display_address": [
                    "Blumenbergplatz",
                    "9000 Saint Gallen",
                    "Switzerland"
                ]
            },
            "phone": "+41712242478",
            "display_phone": "+41 71 224 24 78",
            "distance": 538.8711432373677
        },
        .
        .
        .
    ],
    "total": 3,
    "region": {
        "center": {
            "longitude": 9.369902,
            "latitude": 47.42318
        }
    }
}
```

**STEPS:**

Implement a **function** `find_restaurants` with the following parameters:
- `x`: latitude of a location (`float`)
- `y`: longitude of a location (`float`)
- `open_at`:  datetime when the retrieved restaurants should be open, in the format of Unix timestamp (`int`)
- `radius`: the maximum distance of the discovered restaurants from a location (`int`) 

The function should send a request to the Yelp API for retrieving  businesses of the category "restaurants" which are a) located within `radius` of (`x`,`y`), and b) `open_at` the specified datetime. 

Please note that the parameter `open_at` only works for values that are in the same week (Monday to Sunday) as the current one. This means, when you try to run the code on a Sunday the API might not return anything. But if you run the same code on Monday again, it should work. This is a limitation of the API. See this GitHub issue for more information: [github.com/Yelp/yelp-fusion/issues/371](https://github.com/Yelp/yelp-fusion/issues/371) 

The function should use the retrieved restaurants and return a `list` of `tuples`. Each tuple should contain the **name** and **rating** of a restaurant.
    
You can find more hints at the end of this document.

In [ ]:
# PUT YOUR API KEY HERE

# Add your code here:




In [ ]:
# Run this cell to try out your function:
# Call the function to get the restaurants that are close to Zurich HB

# open now
t1 = int(time.time())
restaurants_1 = find_restaurants(47.377847, 8.540502, t1, 500)
print(restaurants_1)

# open in 12 hours
t2 = t1 + 12 * 60 * 60
restaurants_2 = find_restaurants(47.377847, 8.540502, t2, 700)
print(restaurants_2)

In [ ]:
# Test Cell.
from unittest import TestCase
from inspect import signature
from datetime import datetime, timedelta
import time

__ = TestCase()

__.assertTrue("find_restaurants" in locals(), msg='You did not define a function called find_restaurants.')
__.assertEqual(len(signature(find_restaurants).parameters), 4, msg='Your function does not have exactly 4 parameters.')

datetime_tomorrow = datetime.now() + timedelta(days=1)
morning_datetime = datetime_tomorrow.replace(hour=7)
evening_datetime = morning_datetime.replace(hour=20)

# Get the UNIX time.
morning_timestamp = int((morning_datetime - datetime(1970, 1, 1)).total_seconds())
evening_timestamp = int((evening_datetime - datetime(1970, 1, 1)).total_seconds())

t22_return = find_restaurants(47.377847, 8.540502, morning_timestamp, 500)
t22_return_evening = find_restaurants(47.377847, 8.540502, evening_timestamp, 500)

__.assertIsInstance(t22_return, list, msg='The returned value should be a list.')
__.assertTrue(all(isinstance(t, tuple) for t in t22_return), msg='The values of the list should be tuples.')
__.assertTrue(all(len(t) == 2  for t in t22_return), msg='Each tuple should contain exactly 2 values, representing the name and rating of a restaurant.')
__.assertTrue(all(isinstance(t[0], str) for t in t22_return), msg='The 1st value in a tuple should be a string, representing the name of a reastaurant.')
__.assertTrue(all(isinstance(t[1], float) for t in t22_return), msg='The 2nd value in a tuple should be a float, representing the ranking of a reastaurant.')

__.assertTrue(all(t[1] >= 0 for t in t22_return), msg='All rating values should be greater than 0.')
__.assertTrue(all(t[1] <= 5 for t in t22_return), msg='All rating values should be less than 5.')
__.assertIn(('Sprüngli', 4.7), t22_return, msg = 'The function does not return the correct value.' )
__.assertNotIn(('Sprüngli', 4.7), t22_return_evening, msg = 'The function does not return the correct value. You found a restaurant that should be closed.' )
__.assertNotIn(('Henrici', 4.0), t22_return, msg = 'The function does not return the correct value. You found a restaurant that exceeds the radius.')

print("\n\033[37;42;2m  Success! Your code works as intended.  \033[0m\n")


### Task 2.3: Implement a Web Mashup for finding restaurants during trips
    
**GOAL: Implement a Web Mashup that finds the best restaurants next to your destination.**
In this Task, you are going to combine the functionalities (i.e., properly call the functions) of Task 2.1 and Task 2.2 to:
1. Provide an origin location, an arrival location and a datetime for a trip, and
2. Retrieve the best-rated restaurants that are within a given radius of the arrival location.

**STEPS**:

Implement a **function** `find_top10_restaurants_for_trip` with the following parameters: 
  - `origin`: origin location (`string`)
  - `destination`: arrival location (`string`)
  - `departure_date` : date of departure, in the format YYYY-MM-DD (`string`) 
  - `departure_time` : time of departure, in the format in hh:mm (`string`)
  - `radius`: maximum distance (meters) of the retrieved restaurants from a location (`int`)

The function should return a `list` of `tuples` with names and rankings about the **10** retrieved restaurants with the **highest rankings in descending order**. The restaurants should be within a `radius` of the trip `destination`, and `open at` the arrival datetime. 

For implementing the functionality of `find_top10_restaurants_for_trip`, call your functions from Tasks 2.1 and 2.2.

In [ ]:
# Add your code here:




In [ ]:
# Run this cell to try out your function
ch_datetime = datetime.now() + timedelta(hours=2)
depart_date = ch_datetime.strftime("%Y-%m-%d")
depart_time = ch_datetime.strftime("%H:%M")

#Call your function to get the top 10 restaurants for your upcoming trips

# From St. Gallen to Zurich
restaurants_1 = find_top10_restaurants_for_trip("St. Gallen", "Zurich", depart_date, depart_time, 500)
print(restaurants_1)

# From Zurich to St. Gallen
restaurants_2 = find_top10_restaurants_for_trip("Zurich", "St. Gallen", depart_date, depart_time, 1000)
print(restaurants_2)

In [ ]:
# Test Cell.
from unittest import TestCase
from inspect import signature
from datetime import datetime
import time

__ = TestCase()

# Signature
__.assertTrue("find_top10_restaurants_for_trip" in locals(), msg='You did not define a function called find_top10_restaurants_for_trip.')
__.assertEqual(len(signature(find_top10_restaurants_for_trip).parameters), 5, msg='Your function does not have exactly 5 parameters.')

datetime_tomorrow = datetime.now() + timedelta(days=1)
morning_datetime = datetime_tomorrow.replace(hour = 7)
evening_datetime = morning_datetime.replace(hour = 20)

t23_return = find_top10_restaurants_for_trip("St. Gallen", "Zurich", morning_datetime.strftime("%Y-%m-%d"), morning_datetime.strftime("%H:%M"),500)
t23_return_evening =find_top10_restaurants_for_trip("St. Gallen", "Zurich", evening_datetime.strftime("%Y-%m-%d"), evening_datetime.strftime("%H:%M"),500)

# Datatypes 
__.assertIsInstance(t23_return, list, msg='The returned value should be a list.')
__.assertTrue(all(isinstance(t, tuple) for t in t23_return), msg='The values of the list should be tuples.')
__.assertTrue(all(len(t) == 2  for t in t23_return), msg='Each tuple should contain exactly 2 values, representing the name and rating of a restaurant.')
__.assertTrue(all(isinstance(t[0], str) for t in t23_return), msg='The 1st value in a tuple should be a string, representing the name of a reastaurant.')
__.assertTrue(all(isinstance(t[1], float) for t in t23_return), msg='The 2nd value in a tuple should be a float, representing the ranking of a reastaurant.')

# Values
__.assertTrue(all(t[1] >= 0 for t in t23_return), msg='All rating values should be greater than 0.')
__.assertTrue(all(t[1] <= 5 for t in t23_return), msg='All rating values should be less than 5.')
__.assertIn(('Sprüngli', 4.7), t23_return, msg = 'The function does not return the correct value.' )
__.assertNotIn(('Pizzeria Atrio', 3.0), t23_return, msg = 'The function does not return the correct value.' )
__.assertTrue(all(t23_return[i][1]>=t23_return[i+1][1]  for i in range(len(t23_return)-1)), msg='The returned list is not ordered in descending order.')

print("\n\033[37;42;2m  Success! Your code works as intended.  \033[0m\n")

<a class="anchor" name="hints"></a>
## HINTS

**Task 2.1**
- Visit the [documentation of the Transport API](https://transport.opendata.ch/docs.html#connection) to **decide which parameters to use** for querying the connections from an origin to a destination at a specific datetime. You will use these parameters to define the **request parameters**.

- Visit the [documentation of the requests library](https://requests.kennethreitz.org/en/master/user/quickstart/#passing-parameters-in-urls) to find out **a)** how to **define the request parameters**, and **b)** how to **send an HTTP GET request** using the request parameters and the `url` http://transport.opendata.ch/v1/connections.

- Using [json](https://docs.python.org/3/library/json.html), print and observe the **response payload**, and convert it to a Python dictionary: **Access the data** of the 1st connection and, specifically, the necessary values of its nested dictionaries. The goal is to access and return the latitude, the longitude and the timestamp of the arrival.

**Task 2.2**
- Visit the [documentation of the Yelp Fusion API](https://www.yelp.com/developers/documentation/v3/business_search) to **decide which request parameters to use** for querying `restaurants` that are a) `open at` a specific time and b) close to a location (`latitude`,`longitude`) whithin a specific `radius`. 

- Visit the [documentation of the requests library](https://requests.kennethreitz.org/en/master/user/quickstart/#passing-parameters-in-urls) to find out **a)** [how to **construct a custom header**](https://requests.readthedocs.io/en/master/user/quickstart/#custom-headers), **b** how to **define request parameters**, and c) how to **send an HTTP GET request** using the `header`, the `parameters` and the `url` https://api.yelp.com/v3/businesses/search.

- Using [json](https://docs.python.org/3/library/json.html), print and observe the **response payload**, and convert it to a Python dictionary: **Access the data** of the businesses. The goal is to return a list that contains one tuple for each restaurant with its name and rating. 


<a class="anchor" id="apilist"></a>
## FURTHER API LINKS

The Transport API and the Yelp Fusion API you worked with in this assignment are only two exampes of public APIs where you can get data from.
Here, we provide you with a list of publicly available APIs that might be interesting (e.g., for your group project).
Before you use any API make sure that you trust the services and that you are aware of their pricing scheme if they are not free to use!

**Public APIs from Switzerland**
- Swiss Open Government data (>10.000 datasets, [Link](https://opendata.swiss/en), API Documentation in [German](https://handbook.opendata.swiss/de/content/nutzen/api-nutzen.html) or [French](https://handbook.opendata.swiss/fr/content/nutzen/api-nutzen.html))
- Switzerland Tourism | OpenData API ([Link](https://developer.myswitzerland.io/), [API overview](https://swagger.myswitzerland.io/))
- Open data platform mobility Switzerland ([Link](https://opentransportdata.swiss/en/))
- SBB Open Data ([Link](https://data.sbb.ch))
- Swiss GeoAdmin API ([Link](https://www.geo.admin.ch/en/programming-interface-api), [API documentation](https://api3.geo.admin.ch/))
- SRG SSR APIs ([Link](https://developer.srgssr.ch/api-catalog))
- Swiss Post ([Link](https://developer.apis.post.ch/ui/home))
- Open Food Data CH (List of multiple APIs, e.g. [Swiss Food Composition Database](https://food.schoolofdata.ch/food-composition-ch/), [Link](https://food.opendata.ch))

**Other Countries**

Open, public APIs are released by many governments around the world. Here are some examples:
- Austria Open Data ([Link](https://www.data.gv.at/en/))
- Germany ([Link](https://bund.dev), in German)
- UK ([Link](https://www.api.gov.uk))
- France ([Link](https://api.gouv.fr/), in French)
- USA ([Link](https://open.gsa.gov/))
- India ([Link](https://data.gov.in/))
- European Union ([Link](https://data.europa.eu/en), [API Reference](https://data.europa.eu/en/which-apis-are-available-and-where-can-i-find-information-about-them))

**OpenAI**

Open AI lets you access the ChatGPT and Whisper (Speech-to-Text) via an API ([Link](https://openai.com/blog/introducing-chatgpt-and-whisper-apis), [API reference](https://platform.openai.com/docs/api-reference)). This can be very helpful when you want to deal with natural language input or output.

**API Directories**
- Public APIs ([Link](https://publicapis.io/))
- Public API Lists ( [Link](https://github.com/public-api-lists/public-api-lists))
- APIs Guru ([Link](https://apis.guru/))
- Public APIs ([Link](https://publicapis.dev/))
- Rapic API ([Link](https://rapidapi.com/hub))
- Postman API Network ([Link](https://www.postman.com/explore))